In [1]:
from darts.metrics import mape
from darts import TimeSeries
import pandas as pd
import seaborn as sns
import numpy as np
import lightgbm as lgb
from tqdm import tqdm

In [2]:
df = pd.read_parquet('data/clean/df.parquet').drop(columns=['next_hour_forecast'])
df.head(3)

,next_hour_load
datetime,
2014-12-15 00:00:00,6131.0
2014-12-15 01:00:00,5842.0
2014-12-15 02:00:00,5715.0


# Build baseline

Use the load from 24h ago

In [3]:
# Enrich the df with the load 24h ago
df['24h_ago_load'] = df.shift(24).next_hour_load
df = df.dropna()
df.head(3)

,next_hour_load,24h_ago_load
datetime,,
2014-12-16 00:00:00,6624.0,6131.0
2014-12-16 01:00:00,6292.0,5842.0
2014-12-16 02:00:00,6255.0,5715.0


In [4]:
# Split train:val
datetime_cutoff = pd.Timestamp('2024-08-01')
val_df = df[df.index >= datetime_cutoff]

# Compute MAPE
val_gt_ts = TimeSeries.from_dataframe(val_df, value_cols=['next_hour_load'], freq='h')
val_forecast_ts = TimeSeries.from_dataframe(val_df, value_cols=['24h_ago_load'], freq='h')

print('Val MAPE: ', mape(val_gt_ts, val_forecast_ts))

Val MAPE:  7.7380574999523715


# Build smarter baseline

Use the load from 24h ago + the datetime attribute (day, month, hour, weekday) as features for LGBM

In [5]:
# Enrich the df with the datetime attributes
df['month'] = df.index.month
df['day'] = df.index.day
df['hour'] = df.index.hour
df['weekday'] = df.index.weekday
df.head(3)

,next_hour_load,24h_ago_load,month,day,hour,weekday
datetime,,,,,,
2014-12-16 00:00:00,6624.0,6131.0,12,16,0,1
2014-12-16 01:00:00,6292.0,5842.0,12,16,1,1
2014-12-16 02:00:00,6255.0,5715.0,12,16,2,1


In [6]:
# Build Xy
Xy = df[['month', 'day', 'hour', 'weekday', '24h_ago_load', 'next_hour_load']]

# Split train:val
datetime_cutoff = pd.Timestamp('2024-08-01')
Xy_train = Xy[Xy.index < datetime_cutoff]
Xy_val = Xy[Xy.index >= datetime_cutoff]

# Split X,y
X_train, y_train = Xy_train.drop(columns=['next_hour_load']), Xy_train.next_hour_load
X_val, y_val = Xy_val.drop(columns=['next_hour_load']), Xy_val.next_hour_load

In [7]:
reg = lgb.LGBMRegressor(n_estimators=100, force_row_wise=True)
reg.fit(X_train, y_train)

[LightGBM] [Info] Total Bins 331
[LightGBM] [Info] Number of data points in the train set: 84070, number of used features: 5
[LightGBM] [Info] Start training from score 7103.894957


LGBMRegressor(force_row_wise=True)

In [8]:
# Compute MAPE
y_train_ts = TimeSeries.from_values(y_train)
yhat_train_ts = TimeSeries.from_values(reg.predict(X_train))

y_val_ts = TimeSeries.from_values(y_val)
yhat_val_ts = TimeSeries.from_values(reg.predict(X_val))

print('Train MAPE:', mape(y_train_ts, yhat_train_ts))
print('Val MAPE:', mape(y_val_ts, yhat_val_ts))

Train MAPE: 4.083923867893747
Val MAPE: 7.810632861740726


# Backtesting

In real life, each model is trained with all the historical data available, and the load prediction for in 24h is then made.

Hence, we need to build some backtesting. 
For each timestamp, a new model will be trained. 
It will then be used to predict the load in 24h.

In [10]:
def backtesting(Xy, model, starting_ts=pd.Timestamp('2024-08-01'), use_every_nth_ts=1):
    cutoff_ts = Xy[Xy.index >= starting_ts].index.to_list()
    
    cutoff_ts_to_y = {}
    for ts in tqdm(cutoff_ts[::use_every_nth_ts]):    
        
        # Split train:val
        Xy_train = Xy[Xy.index < ts]
        Xy_val = Xy[Xy.index == ts]
        
        # Split X,y
        X_train, y_train = Xy_train.drop(columns=['next_hour_load']), Xy_train.next_hour_load
        X_val, y_val = Xy_val.drop(columns=['next_hour_load']), Xy_val.next_hour_load
    
        # Train model
        model.fit(X_train, y_train)
    
        # Compute prediction in 24h
        yhat_val = model.predict(X_val) 
    
        cutoff_ts_to_y[ts] = (yhat_val[0], y_val.iloc[0])
        
    return pd.DataFrame({
        'cutoff_ts': cutoff_ts_to_y.keys(), 
        'predicted_next_hour_load': [e[0] for e in cutoff_ts_to_y.values()], 
        'next_hour_load': [e[1] for e in cutoff_ts_to_y.values()]
    })
        

In [11]:
reg = lgb.LGBMRegressor(n_estimators=100, force_row_wise=True, verbose=0)
results_df = backtesting(Xy, model=reg, starting_ts=pd.Timestamp('2024-08-01'), use_every_nth_ts=1)
results_df.head(3)

100%|███████████████████████████████████████| 1234/1234 [03:23<00:00,  6.05it/s]


,cutoff_ts,predicted_next_hour_load,next_hour_load
0,2024-08-01 00:00:00,5413.437410,5219.0
1,2024-08-01 01:00:00,5253.883738,4820.0
2,2024-08-01 02:00:00,5177.821791,4550.0


In [12]:
print(f'Backtested MAPE: {
    mape(
        TimeSeries.from_values(results_df.predicted_next_hour_load),
        TimeSeries.from_values(results_df.next_hour_load)
    )
}')

Backtested MAPE: 6.500857665725692


# Add last week's load

As feature, use 
- Load 24h ago
- Load a week ago
- Datetime attributes

In [15]:
# Enrich the df with the load 192h ago (i.e. 8 days)
df['8d_ago_load'] = df.shift(24*8).next_hour_load
df = df.dropna()
df.head(3)

,next_hour_load,24h_ago_load,month,day,hour,weekday,8days_ago_load,8d_ago_load
datetime,,,,,,,,
2015-01-11 00:00:00,6703.0,7075.0,1,11,0,6,7100.0,7100.0
2015-01-11 01:00:00,6433.0,6798.0,1,11,1,6,6924.0,6924.0
2015-01-11 02:00:00,6419.0,6701.0,1,11,2,6,6916.0,6916.0


In [17]:
# Build Xy
Xy = df[[
    'month', 'day', 'hour', 'weekday', 
    '24h_ago_load', 
    '8d_ago_load',
    'next_hour_load'
]]
Xy.head(3)

In [18]:
reg = lgb.LGBMRegressor(n_estimators=100, force_row_wise=True, verbose=0)
results_df = backtesting(Xy, model=reg, starting_ts=pd.Timestamp('2024-08-01'), use_every_nth_ts=1)
results_df.head(3)

100%|███████████████████████████████████████| 1234/1234 [03:44<00:00,  5.50it/s]


,cutoff_ts,predicted_next_hour_load,next_hour_load
0,2024-08-01 00:00:00,5458.378034,5219.0
1,2024-08-01 01:00:00,5208.402894,4820.0
2,2024-08-01 02:00:00,5102.320114,4550.0


In [19]:
print(f'Backtested MAPE: {
    mape(
        TimeSeries.from_values(results_df.predicted_next_hour_load),
        TimeSeries.from_values(results_df.next_hour_load)
    )
}')

Backtested MAPE: 6.217547513280606


# Add [-48h;-24h] and [-8d, -1d] statistics

As features, start using
- Min/Max/Median load 

In [20]:
def statistic_load_nhours_to_24_hours_ago(current_time, start_n_hours_ago, stat):
    start_time = current_time - pd.Timedelta(hours=start_n_hours_ago)
    end_time = current_time - pd.Timedelta(hours=24)
    
    relevant_data = df.loc[start_time:end_time, 'next_hour_load']

    if len(relevant_data) == 0:
        return np.nan
    
    return stat(relevant_data.values)

In [21]:
# Compute median
df['previous_day_median'] = df.index.to_series().apply(lambda x: statistic_load_nhours_to_24_hours_ago(x, start_n_hours_ago=48, stat=np.median))
df['previous_week_median'] = df.index.to_series().apply(lambda x: statistic_load_nhours_to_24_hours_ago(x, start_n_hours_ago=24*7 + 24, stat=np.median))
df['previous_month_median'] = df.index.to_series().apply(lambda x: statistic_load_nhours_to_24_hours_ago(x, start_n_hours_ago=24*30 + 24, stat=np.median))
df.head(3)

,next_hour_load,24h_ago_load,month,day,hour,weekday,8d_ago_load,previous_day_median,previous_week_median,previous_month_median
datetime,,,,,,,,,,
2015-01-11 00:00:00,6703.0,7075.0,1,11,0,6,7100.0,NaN,NaN,NaN
2015-01-11 01:00:00,6433.0,6798.0,1,11,1,6,6924.0,NaN,NaN,NaN
2015-01-11 02:00:00,6419.0,6701.0,1,11,2,6,6916.0,NaN,NaN,NaN


In [22]:
# Compute min
df['previous_day_min'] = df.index.to_series().apply(lambda x: statistic_load_nhours_to_24_hours_ago(x, start_n_hours_ago=48, stat=np.min))
df['previous_week_min'] = df.index.to_series().apply(lambda x: statistic_load_nhours_to_24_hours_ago(x, start_n_hours_ago=24*7 + 24, stat=np.min))
df['previous_month_min'] = df.index.to_series().apply(lambda x: statistic_load_nhours_to_24_hours_ago(x, start_n_hours_ago=24*30 + 24, stat=np.min))
df.head(3)

,next_hour_load,24h_ago_load,month,day,hour,weekday,8d_ago_load,previous_day_median,previous_week_median,previous_month_median,previous_day_min,previous_week_min,previous_month_min
datetime,,,,,,,,,,,,,
2015-01-11 00:00:00,6703.0,7075.0,1,11,0,6,7100.0,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-11 01:00:00,6433.0,6798.0,1,11,1,6,6924.0,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-11 02:00:00,6419.0,6701.0,1,11,2,6,6916.0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Compute max
df['previous_day_max'] = df.index.to_series().apply(lambda x: statistic_load_nhours_to_24_hours_ago(x, start_n_hours_ago=48, stat=np.max))
df['previous_week_max'] = df.index.to_series().apply(lambda x: statistic_load_nhours_to_24_hours_ago(x, start_n_hours_ago=24*7 + 24, stat=np.max))
df['previous_month_max'] = df.index.to_series().apply(lambda x: statistic_load_nhours_to_24_hours_ago(x, start_n_hours_ago=24*30 + 24, stat=np.max))
df.head(3)

,next_hour_load,24h_ago_load,month,day,hour,weekday,8d_ago_load,previous_day_median,previous_week_median,previous_month_median,previous_day_min,previous_week_min,previous_month_min,previous_day_max,previous_week_max,previous_month_max
datetime,,,,,,,,,,,,,,,,
2015-01-11 00:00:00,6703.0,7075.0,1,11,0,6,7100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-11 01:00:00,6433.0,6798.0,1,11,1,6,6924.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-11 02:00:00,6419.0,6701.0,1,11,2,6,6916.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Build Xy
Xy = df[[
    'month', 'day', 'hour', 'weekday', 
    '24h_ago_load', 
    '8d_ago_load', 
    'previous_day_median', 'previous_week_median', 'previous_month_median',
    'previous_day_min', 'previous_week_min', 'previous_month_min', 
    'previous_day_max', 'previous_week_max', 'previous_month_max',
    'next_hour_load'
]]
Xy.head(3)

,month,day,hour,weekday,24h_ago_load,8d_ago_load,previous_day_median,previous_week_median,previous_month_median,previous_day_min,previous_week_min,previous_month_min,previous_day_max,previous_week_max,previous_month_max,next_hour_load
datetime,,,,,,,,,,,,,,,,
2015-01-11 00:00:00,1,11,0,6,7075.0,7100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6703.0
2015-01-11 01:00:00,1,11,1,6,6798.0,6924.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6433.0
2015-01-11 02:00:00,1,11,2,6,6701.0,6916.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6419.0


In [25]:
reg = lgb.LGBMRegressor(n_estimators=100, force_row_wise=True, verbose=0)
results_df = backtesting(Xy, model=reg, starting_ts=pd.Timestamp('2024-08-01'), use_every_nth_ts=1)
results_df.head(3)

100%|███████████████████████████████████████| 1234/1234 [07:05<00:00,  2.90it/s]


,cutoff_ts,predicted_next_hour_load,next_hour_load
0,2024-08-01 00:00:00,5240.421279,5219.0
1,2024-08-01 01:00:00,5004.318443,4820.0
2,2024-08-01 02:00:00,4999.368395,4550.0


In [26]:
print(f'Backtested MAPE: {
    mape(
        TimeSeries.from_values(results_df.predicted_next_hour_load),
        TimeSeries.from_values(results_df.next_hour_load)
    )
}')

Backtested MAPE: 5.646689801941207


# Try with a bigger # estimators

In [33]:
reg = lgb.LGBMRegressor(n_estimators=10_000, force_row_wise=True, verbose=0)
results_df = backtesting(Xy, model=reg, starting_ts=pd.Timestamp('2024-08-01'), use_every_nth_ts=100)
results_df.head(3)

100%|███████████████████████████████████████████| 13/13 [02:55<00:00, 13.46s/it]


,cutoff_ts,predicted_next_hour_load,next_hour_load
0,2024-08-01 00:00:00,5550.856580,5219.0
1,2024-08-05 04:00:00,4452.793656,4583.0
2,2024-08-09 08:00:00,5133.093790,5298.0


In [34]:
print(f'Backtested MAPE: {
    mape(
        TimeSeries.from_values(results_df.predicted_next_hour_load),
        TimeSeries.from_values(results_df.next_hour_load)
    )
}')

Backtested MAPE: 4.273615743935125


# Ideas

- More robust hyperparameters search (optuna)
- Use weather data as covariate and future covariates
- Use the energy outage data

## Notes
- The data is notoriously NOT accurate, and hence shouldn't be treated as such


## TODO

Build website to showcase project
- Landing page shows a pretty plot with the historical data and prediction
- One page about data exploration (EDA), showing plots, seasonality, etc.
- One page about the modelling